In [21]:
import pandas as pd
import numpy as np
import seaborn as sns

In [22]:
dat1 = pd.read_csv("3rd_ppp_for_upload_win.csv", encoding = "ISO-8859-1")
dat2 = pd.read_csv("atm_for_upload_win.csv", encoding = "ISO-8859-1")
dat3 = pd.read_csv("banks_for_upload_win.csv", encoding = "ISO-8859-1")
dat4 = pd.read_csv("bus_stands_for_upload_win.csv", encoding = "ISO-8859-1")
dat5 = pd.read_csv("microfinance_for_upload_win.csv", encoding = "ISO-8859-1")
dat6 = pd.read_csv("mobilemoney_agents_for_upload_win.csv", encoding = "ISO-8859-1")
dat7 = pd.read_csv("pos_for_upload_win.csv", encoding = "ISO-8859-1")
dat8 = pd.read_csv("post_office_for_upload_win.csv", encoding = "ISO-8859-1")
dat9 = pd.read_csv("sacco_for_upload_win.csv", encoding = "ISO-8859-1")

C:\Users\dell\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (26,32,33,42,74) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [23]:

dat1 = dat1[['district', 'latitude', 'longitude', 'agent_name',
       'standalone_agent', 'trading_hours']]

In [24]:
dat1.drop(["trading_hours",'agent_name','district'], axis = 1, inplace =True)

In [25]:
dat1['standalone_agent'] = dat1['standalone_agent'].astype('category')

In [26]:
data = dat1.copy()

In [27]:
cat_columns = dat1.select_dtypes(['category']).columns

cat_columns
dat1[cat_columns] = dat1[cat_columns].apply(lambda x: x.cat.codes)
dat1.head()

,latitude,longitude,standalone_agent
0,-2.36921,33.57654,0
1,-2.53201,32.94780,0
2,-6.84188,39.21102,1
3,-6.77949,39.24092,0
4,-6.82792,39.31021,1


In [28]:

X= dat1.drop('standalone_agent', axis =1)
y = dat1["standalone_agent"]

In [29]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=42)

In [30]:
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [31]:

model = LogisticRegression()
rf = RandomForestClassifier()
xgb = XGBClassifier()

In [32]:
xgb.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [34]:
from sklearn.metrics import accuracy_score

In [35]:
accuracy_score(xgb.predict(X_test),y_test)

0.6357487922705314

In [36]:

xgb.fit(X,y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [37]:
train = pd.read_csv("training.csv")
test = pd.read_csv("test.csv")

In [39]:
train  = train[["Latitude","Longitude"]]
test = test[["Latitude","Longitude"]]

In [40]:

train.columns = ['latitude', 'longitude']
test.columns = ['latitude', 'longitude']

In [41]:
train_trade = xgb.predict(train)

In [42]:
test_trade = xgb.predict(test)

In [43]:
mainmain = pd.concat([dat1.standalone_agent,data.standalone_agent], axis = 1)
mainmain.head()

,standalone_agent,standalone_agent
0,0,Conduct other business
1,0,Conduct other business
2,1,Stand Alone Agent
3,0,Conduct other business
4,1,Stand Alone Agent


In [44]:

mainmain.columns = ['standing', 'standalone_agent']

In [45]:

#worked_out.head()
dict(zip(mainmain.standing, mainmain.standalone_agent))

{0: 'Conduct other business', 1: 'Stand Alone Agent'}

In [46]:
dick = dict(zip(mainmain.standing, mainmain.standalone_agent)) 
dick

{0: 'Conduct other business', 1: 'Stand Alone Agent'}

In [47]:
trainpred = pd.DataFrame(train_trade, columns = ["standalone_agent"])
testpred = pd.DataFrame(test_trade ,  columns = ["standalone_agent"])

In [48]:
trainagent = trainpred.replace({'standalone_agent': dick})
trainagent.head()

,standalone_agent
0,Conduct other business
1,Conduct other business
2,Conduct other business
3,Conduct other business
4,Stand Alone Agent


In [49]:

testagent = testpred.replace({'standalone_agent': dick})
testagent.head()

,standalone_agent
0,Conduct other business
1,Conduct other business
2,Conduct other business
3,Conduct other business
4,Conduct other business


In [50]:

trainagent.to_csv('trainagent.csv',index = False)
testagent.to_csv('testagent.csv', index = False)